In [28]:
import pandas as pd

In [29]:
from plotly.offline import iplot
import cufflinks as cf
cf.go_offline()

In [30]:
symbol = 'aapl'

In [31]:
data = pd.read_csv('data/pred/pred_sto_rea_twt_' + symbol + '.csv', index_col=0, parse_dates=True)[1:]
prices = pd.read_csv('data/prices_open/prices_open_' + symbol + '.csv', index_col=0, parse_dates=True)[1:]

In [32]:
(prices.price_open[-1] - prices.price_open[0]) / prices.price_open[0]

0.16819606952920402

In [33]:
th = 0.6
dec_history = []
capital = 100000
assets = 0
curr = pd.Series()
for s, p in zip(data.pred[:-1], prices.price_open):
    decision = 0
    if s > curr.quantile(th):
        decision = 1
    if s < curr.quantile(1-th):
        decision = -1

    if decision == 1 and capital > 0:
        assets = capital / p
        capital = 0
    elif decision == -1 and assets > 0:
        capital = assets * p
        assets = 0

    dec_history.append([s, decision, capital, p, assets, (max(capital, assets * p) - 100000) / 100000, (p - prices.price_open[0]) / prices.price_open[0]])
    curr = curr.append(pd.Series(s))
final = max(capital, assets * prices.price_open[-1]) 
final = (final - 100000) / 100000

In [34]:
final

0.26288826832679596

In [27]:
pd.DataFrame(dec_history, columns=['sent', 'decision', 'capital', 'price', 'assets', 'o', 'm']).replace({'decision' : {1 : 'buy', -1 : 'sell'}}).round(2).to_csv('tmp_dec_history.csv')